In [117]:
import pandas as pd

import plotly.express as pe


#model

from sklearn.cluster import KMeans

#metrics

from sklearn.metrics import silhouette_score, calinski_harabasz_score

### I want to market high risk high rewards mutual funds investment plan
### The plan requires heavy investment every year
### the plan is aimed at individuals who have more free cash available and can invest for a lot of years

In [118]:
path = r"/home/harshit/Desktop/IntroductionToML/Dataset/original.csv"

df = pd.read_csv(path)

df

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


### step 2: Data exploration

In [119]:
df.shape

(2000, 5)

In [120]:
df.index

RangeIndex(start=0, stop=2000, step=1)

In [121]:
df.columns

Index(['clientid', 'income', 'age', 'loan', 'default'], dtype='object')

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  2000 non-null   int64  
 1   income    2000 non-null   float64
 2   age       1997 non-null   float64
 3   loan      2000 non-null   float64
 4   default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.3 KB


In [123]:
df.isna().sum()

clientid    0
income      0
age         3
loan        0
default     0
dtype: int64

In [124]:
df.nunique()

clientid    2000
income      2000
age         1997
loan        2000
default        2
dtype: int64

In [125]:
df["default"].value_counts()

0    1717
1     283
Name: default, dtype: int64

### features

In [126]:
real_value_features = ["loan", "age", "income"]

In [127]:
df[     real_value_features     ].describe()

,loan,age,income
count,2000.000000,1997.000000,2000.000000
mean,4444.369695,40.807559,45331.600018
std,3045.410024,13.624469,14326.327119
min,1.377630,-52.423280,20014.489470
25%,1939.708847,28.990415,32796.459717
50%,3974.719419,41.317159,45789.117313
75%,6432.410625,52.587040,57791.281668
max,13766.051239,63.971796,69995.685578


## confirm outliers

In [128]:
for col in real_value_features :
    fig = pe.box(   y=col, data_frame=df  )
    display(fig)

### step 2b) Preprocessing the data!

In [129]:
condition1 = ( df["age"] >= 0)
condition2 = ( df["loan"] < 1317268)
data = df[    condition1 & condition2    ].copy() #makes new table "data"

del df #delete reference to original table

data.dropna(inplace=True) #drop missing values. DO THIS INPLACE meaning in the same object!

data.reset_index(drop=True, inplace=True)

### cluster this data in such a way that income and age become factors of similarity


In [130]:
data[real_value_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   loan    1994 non-null   float64
 1   age     1994 non-null   float64
 2   income  1994 non-null   float64
dtypes: float64(3)
memory usage: 46.9 KB


In [131]:
data

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1989,1996,59221.044874,48.518179,1926.729397,0
1990,1997,69516.127573,23.162104,3503.176156,0
1991,1998,44311.449262,28.017167,5522.786693,1
1992,1999,43756.056605,63.971796,1622.722598,0


possible payment years: 100 - age (round to integer)
free - cash : (income/12) - loan

In [132]:
data["possible_payment_term"]=100-data["age"]

data["free_cash"] = (data["income"] / 12) - ( data["loan"] / 12)

data

,clientid,income,age,loan,default,possible_payment_term,free_cash
0,1,66155.925095,59.017015,8106.532131,0,40.982985,4837.449414
1,2,34415.153966,48.117153,6564.745018,0,51.882847,2320.867412
2,3,57317.170063,63.108049,8020.953296,0,36.891951,4108.018064
3,4,42709.534201,45.751972,6103.642260,0,54.248028,3050.490995
4,5,66952.688845,18.584336,8770.099235,1,81.415664,4848.549134
...,...,...,...,...,...,...,...
1989,1996,59221.044874,48.518179,1926.729397,0,51.481821,4774.526290
1990,1997,69516.127573,23.162104,3503.176156,0,76.837896,5501.079285
1991,1998,44311.449262,28.017167,5522.786693,1,71.982833,3232.388547
1992,1999,43756.056605,63.971796,1622.722598,0,36.028204,3511.111167


# what are the right number of clusters?

In [133]:
features = ["possible_payment_term", "free_cash"]

ans = []

for count in range(2, 21,1):
    model = KMeans(n_clusters=count, n_init=10)
    model.fit_transform(data[features])
    ans.append(   model.inertia_   )



pe.line(

    x=range(2,21),
    y=ans,
    markers=True
)

### final model

In [134]:
model = KMeans(n_clusters=4, n_init=10)

#1994
values= model.fit_predict(data[features])

predictions = pd.DataFrame(values, columns=["predicted_cluster"])


result_df = pd.concat(   [data, predictions], axis=1    )

result_df["predicted_cluster"] = result_df["predicted_cluster"].astype("object")

result_df

,clientid,income,age,loan,default,possible_payment_term,free_cash,predicted_cluster
0,1,66155.925095,59.017015,8106.532131,0,40.982985,4837.449414,0
1,2,34415.153966,48.117153,6564.745018,0,51.882847,2320.867412,3
2,3,57317.170063,63.108049,8020.953296,0,36.891951,4108.018064,2
3,4,42709.534201,45.751972,6103.642260,0,54.248028,3050.490995,1
4,5,66952.688845,18.584336,8770.099235,1,81.415664,4848.549134,0
...,...,...,...,...,...,...,...,...
1989,1996,59221.044874,48.518179,1926.729397,0,51.481821,4774.526290,0
1990,1997,69516.127573,23.162104,3503.176156,0,76.837896,5501.079285,0
1991,1998,44311.449262,28.017167,5522.786693,1,71.982833,3232.388547,1
1992,1999,43756.056605,63.971796,1622.722598,0,36.028204,3511.111167,2


In [138]:
pe.scatter(
    x="possible_payment_term",
    y="free_cash",
    data_frame=result_df,
    color="predicted_cluster"
)

In [139]:
condition = (result_df["free_cash"].between(5000,6000)) & (result_df["possible_payment_term"].between(70,80))

result_df[condition]

,clientid,income,age,loan,default,possible_payment_term,free_cash,predicted_cluster
58,65,69078.604812,25.107524,4076.583914,0,74.892476,5416.835075,0
77,84,68811.779416,24.038265,4211.302611,0,75.961735,5383.373067,0
106,113,68762.416665,20.991243,2796.752303,0,79.008757,5497.138697,0
111,118,63062.142096,26.585778,56.166164,0,73.414222,5250.497994,0
172,179,67465.062386,23.611054,1802.616994,0,76.388946,5471.870449,0
387,394,65574.093342,23.513043,3031.246326,0,76.486957,5211.903918,0
401,408,65330.192836,28.589987,4030.803692,0,71.410013,5108.282429,0
422,429,62678.645451,25.839394,333.441398,0,74.160606,5195.433671,0
480,487,68052.806919,28.758802,1415.718263,0,71.241198,5553.090721,0
587,594,67984.040377,27.465281,1642.969471,0,72.534719,5528.422576,0


In [137]:
silhouette_score(result_df[features],   result_df["predicted_cluster"] )

0.5580079505968321